In [ ]:
!pip install transformers # Сначала установим библиотеку transformers

In [ ]:
# import
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

import os
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [ ]:
# Устройство
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используется устройство: {DEVICE}")

Используется устройство: cuda


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grafstor/19-000-russian-poems")

print("Path to dataset files:", path)

Using Colab cache for faster access to the '19-000-russian-poems' dataset.
Path to dataset files: /kaggle/input/19-000-russian-poems


In [ ]:
data_path = "/kaggle/input/19-000-russian-poems//poems.csv"

In [ ]:
# Загружаем CSV
dataset = load_dataset("csv", data_files=data_path)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['writer', 'poem', 'text'],
        num_rows: 19316
    })
})

In [ ]:
# 1. Форматируем примеры в Q&A
def format_example(example):
    writer = example.get("writer") or "неизвестный автор"
    poem = example.get("poem") or "без названия"
    text = example.get("text") or ""

    formatted = f'Вопрос: Напиши стихотворение в стиле {writer} под названием "{poem}"\nОтвет: {text.strip()}'
    return {"text": formatted}

dataset = dataset.map(format_example)

Map:   0%|          | 0/19316 [00:00<?, ? examples/s]

In [ ]:
# Загружаем токенизатор и модель
model_name = "ai-forever/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

In [ ]:
# 2. Убираем пустые строки
def filter_empty(example):
    return example["text"] is not None and len(example["text"].strip()) > 0

dataset = dataset.filter(filter_empty)

Filter:   0%|          | 0/19316 [00:00<?, ? examples/s]

In [ ]:
# 3. Токенизация
# Токенизация
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)
# 4. Установка формата для PyTorch
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/19316 [00:00<?, ? examples/s]

In [ ]:
# Коллектор пакетов (динамическое паддинг)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # GPT не использует маскированное моделирование
)

# Аргументы обучения
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,             # попробуй сначала 1 эпоху
    per_device_train_batch_size=2,  # можно увеличить при большем GPU
    per_device_eval_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=200,
    # evaluation_strategy="steps",
    eval_steps=500,
    fp16=torch.cuda.is_available(),  # ускорение на GPU
    report_to="none",                # не слать метрики в wandb
)


In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"] if "test" in tokenized_dataset else None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Запуск обучения
trainer.train()

/tmp/ipython-input-2457577243.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
200,3.802800
400,3.666700
600,3.557200
800,3.560100
1000,3.540300
1200,3.560000
1400,3.454900
1600,3.491000
1800,3.478700
2000,3.487100


TrainOutput(global_step=9658, training_loss=3.402195457889812, metrics={'train_runtime': 4337.0475, 'train_samples_per_second': 4.454, 'train_steps_per_second': 2.227, 'total_flos': 4484695653875712.0, 'train_loss': 3.402195457889812, 'epoch': 1.0})

In [ ]:
prompt = 'Вопрос: Напиши стихотворение в стиле Маяковский под названием "Город" \nОтвет:'
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

output = model.generate(
    **inputs,
    max_length=200,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    do_sample=True
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Вопрос: Напиши стихотворение в стиле Маяковский под названием "Город" 
Ответ: «Ты — город!
Ты — страна!
И ты
Давно уже
Взяла
Поток
В город
К морю
Навсегда!»
Город
Взяла
Поток
В город
К морю
Навсегда.
Море
Взяло
Поток
В город
К морю
Навсегда.
Поток
Взяла
Поток
В город
К морю
Навсегда.
«Мы — люди!
Мы —
К морю
Навсегда!»
Город
Взяла
Поток
В город
К морю
Навсегда.
21 Мая 1919. Iянв.
Коктебель. Город. Iянв. 1920. Iянв. 1920. Iянв. 1921. 1921. Iян. 1922. 1922. Iян.


In [ ]:
# Сохраняем модель и токенизатор
save_path = "./finetuned_rugpt3medium"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("Модель сохранена в:", save_path)

Модель сохранена в: ./finetuned_rugpt3medium


In [ ]:
from google.colab import files
import shutil

shutil.make_archive("finetuned_rugpt3medium", 'zip', save_path)
files.download("finetuned_rugpt3medium.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>